<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GeneticAlgorithmRouting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
from plotly.subplots import make_subplots
np.set_printoptions(formatter={'float':lambda x: f'{x:5.2f}'}, linewidth=120)

In [282]:
N=25
rng=np.random.RandomState(134)
coords = rng.uniform(0,10,size=(N,2))
dist=np.sqrt(np.sum((coords[:,None,:] - coords[None,:,:])**2,axis=-1))

In [283]:
fig=make_subplots()
fig.add_scatter(x=coords[:,0],y=coords[:,1],mode='markers')
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
fig.update_layout(width=600, template='plotly_dark',showlegend=False)

In [406]:
#initial route
seq = np.zeros(N+1, dtype=np.int16)
cur = 0
totdist = 0
rem = list(range(1,N))
seq[0]=cur
for i in range(1,N):
    nxt=rem[np.argmin(dist[cur][rem])]
    totdist += dist[cur][nxt]
    rem.remove(nxt)
    seq[i]=nxt
    cur=nxt
seq[-1]=0
totdist+=dist[cur][0]
bestdist = totdist
bestseq = seq




In [409]:
def plot_single(seq):
    fig=make_subplots()
    fig.add_scatter(x=coords[:,0],y=coords[:,1],mode='markers')
    fig.add_scatter(x=coords[seq,0],y=coords[seq,1],mode='lines')
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.update_layout(width=600, template='plotly_dark',showlegend=False)
    fig.show()

plot_single(seq)

In [414]:
def totdist(seq):
    return np.sum(dist[seq[1:,:],seq[:-1,:]],axis=0)

In [276]:
def mutate(seq, n):
    N, B = seq.shape
    i = np.argsort(rng.uniform(size=(N-2,B)),axis=0)+1
    temp=np.take_along_axis(seq,i[:n],axis=0)
    np.put_along_axis(seq,i[:n], np.take_along_axis(seq,i[n:2*n], axis=0),axis=0)
    np.put_along_axis(seq,i[n:2*n], temp,axis=0)

In [411]:
def mate(seq):
    N, B = seq.shape
    seq=seq[:,rng.permutation(range(B))].reshape(N,2,-1)
    idx=rng.randint(1,N,size=B//2)
    i=np.arange(N)
    children = np.where(i[:,None]>idx[None,:],seq[:,0,:],seq[:,1,:])
    mutate(children[rng.randint(1,N,size=int(0.2*B//2))],1)
    return children

In [417]:
cand=np.tile(ord[:,None],(1,24))
mutate(cand,1)
dists=totdist(cand)
topn=16
topidx=np.argsort(dists)[:topn]

In [418]:
def plot(seqs):
    fig=make_subplots(rows=4,cols=4, shared_xaxes=True, shared_yaxes=True)
    fig.add_scatter(x=coords[:,0], y=coords[:,1],mode='markers')
    for i in range(seqs.shape[1]):
        fig.add_scatter(x=coords[:,0],y=coords[:,1],mode='markers', row=i//4+1,col=i%4 + 1)
        fig.add_scatter(x=coords[seqs[:,i],0],y=coords[seqs[:,i],1],mode='lines',row=i//4+1,col=i%4 + 1)
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.update_layout(width=800, height=800,  template='plotly_dark',showlegend=False)
    fig.show()

In [419]:
plot(cand[:,topidx])